# TP3: Detector de SPAM - Solución

![spam counter](./resources/spam.png)

## Introducción - Detectores de SPAM

Uno de los problemas más comunes en la clasificación es la detección de correos electrónicos SPAM. Uno de los primeros 
modelos utilizados para abordar este problema fue el clasificador de Bayes ingenuo. La detección de SPAM es un problema 
persistente en el mundo digital, ya que los spammers continúan adaptando sus estrategias para eludir los filtros de 
correo no deseado. Además del clasificador de Bayes ingenuo, se han desarrollado y utilizado una variedad de técnicas 
más avanzadas en la detección de SPAM, que incluyen algoritmos de aprendizaje automático, redes neuronales y métodos 
basados en reglas.

En este trabajo práctico, utilizaremos un conjunto de datos que consta de 4601 observaciones de correos electrónicos, 
de los cuales 2788 son correos legítimos y 1813 son correos SPAM. Dado que el contenido de los correos electrónicos es 
un tipo de dato no estructurado, es necesario procesarlo de alguna manera. Para este conjunto de datos, ya se ha 
aplicado un procesamiento típico en el Procesamiento del Lenguaje Natural (NLP), que consiste en contar la frecuencia 
de palabras observadas en los correos.

El procesamiento de lenguaje natural (NLP) desempeña un papel fundamental en la detección de SPAM, ya que permite 
analizar el contenido de los correos electrónicos y extraer características relevantes para la clasificación. Además 
de contar la frecuencia de palabras, se pueden utilizar técnicas más sofisticadas, como la extracción de 
características semánticas y el análisis de sentimientos, para mejorar la precisión de los modelos de detección de SPAM.


Con el fin de preservar la privacidad de los mensajes, la frecuencia de palabras se encuentra normalizada. El conjunto 
de datos está compuesto por 54 columnas de atributos que se denominan:

- `word_freq_XXXX`: Donde `XXXX` es la palabra o símbolo. Los valores son enteros que van de 0 a 20k.

Además, hay una columna adicional llamada `spam`, que es 1 si el correo es SPAM o 0 si no lo es.

## Tareas y preguntas a resolver:

In [18]:
# Importación de librerías.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.naive_bayes import MultinomialNB   
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

In [2]:
# Leemos el dataset
df_spam = pd.read_csv("dataset/spambase.csv")

print(f'Datos obtenidos: ')
df_spam.sample(n=5)

Datos obtenidos: 


,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_edu,word_freq_table,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,spam
4522,0,0,0,0,0,0,0,0,0,0,...,1130,0,0,0,136,0,0,0,409,0
1603,340,170,170,0,1380,690,170,170,0,170,...,0,0,0,0,115,0,0,86,0,1
2981,0,0,1610,0,0,0,0,0,0,0,...,0,0,0,0,283,0,0,0,0,0
3945,0,0,0,0,0,0,0,0,0,620,...,620,0,1860,0,122,122,0,214,0,0
731,170,0,170,170,1440,340,50,50,50,50,...,0,0,0,10,50,0,10,161,30,1


In [3]:
# Chequeamos los tipos de datos.
df_spam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4601 entries, 0 to 4600
Data columns (total 55 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   word_freq_make        4601 non-null   int64
 1   word_freq_address     4601 non-null   int64
 2   word_freq_all         4601 non-null   int64
 3   word_freq_3d          4601 non-null   int64
 4   word_freq_our         4601 non-null   int64
 5   word_freq_over        4601 non-null   int64
 6   word_freq_remove      4601 non-null   int64
 7   word_freq_internet    4601 non-null   int64
 8   word_freq_order       4601 non-null   int64
 9   word_freq_mail        4601 non-null   int64
 10  word_freq_receive     4601 non-null   int64
 11  word_freq_will        4601 non-null   int64
 12  word_freq_people      4601 non-null   int64
 13  word_freq_report      4601 non-null   int64
 14  word_freq_addresses   4601 non-null   int64
 15  word_freq_free        4601 non-null   int64
 16  word_f

In [4]:
# Chequeamos la estadística principal.
df_spam.describe()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_edu,word_freq_table,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,spam
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,104.553358,213.014345,280.656379,65.424908,312.222995,95.900891,114.207564,105.294501,90.067377,239.413171,...,179.823734,5.444469,31.869159,38.574440,139.030428,16.975875,269.068898,75.810259,44.237992,0.394045
std,305.357562,1290.574888,504.142884,1395.151370,672.511666,273.824083,391.440302,401.071452,278.615864,644.755399,...,911.118627,76.274271,285.734646,243.470469,270.355374,109.394164,815.669848,245.879440,429.341596,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,65.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,420.000000,0.000000,380.000000,0.000000,0.000000,0.000000,0.000000,160.000000,...,0.000000,0.000000,0.000000,0.000000,188.000000,0.000000,315.000000,52.000000,0.000000,1.000000
max,4540.000000,14280.000000,5100.000000,42810.000000,10000.000000,5880.000000,7270.000000,11110.000000,5260.000000,18180.000000,...,22050.000000,2170.000000,10000.000000,4385.000000,9752.000000,4081.000000,32478.000000,6003.000000,19829.000000,1.000000


*Notamos el primer problema, o punto a tener en cuenta, los valores máximos, en algunos casos son muy extremos. O sea, hay mails con 40.000 palabras repetidas de por ejemplo: 3d*

In [5]:
# Vemos las cantidades frecuencia de la clase de salida.
df_spam['spam'].value_counts()

spam
0    2788
1    1813
Name: count, dtype: int64

In [6]:
# Chequeamos valores nulos y NaN.
nan_count = df_spam.isnull().sum().sum()
null_count = df_spam.isnull().sum().sum()

print('Valores NaN: ', nan_count)
print('Valores nulos: ', null_count)

Valores NaN:  0
Valores nulos:  0


In [7]:
# Chequeamos la unicicidad de los atributos.
df_spam.nunique()

word_freq_make          142
word_freq_address       171
word_freq_all           214
word_freq_3d             43
word_freq_our           255
word_freq_over          141
word_freq_remove        173
word_freq_internet      170
word_freq_order         144
word_freq_mail          245
word_freq_receive       113
word_freq_will          316
word_freq_people        158
word_freq_report        133
word_freq_addresses     118
word_freq_free          253
word_freq_business      197
word_freq_email         229
word_freq_you           575
word_freq_credit        148
word_freq_your          401
word_freq_font           99
word_freq_000           164
word_freq_money         143
word_freq_hp            395
word_freq_hpl           281
word_freq_george        240
word_freq_650           200
word_freq_lab           156
word_freq_labs          179
word_freq_telnet        128
word_freq_857           106
word_freq_data          184
word_freq_415           110
word_freq_85            177
word_freq_technology

In [8]:
# Chequeamos valores duplicados.
len(df_spam) - len(df_spam.drop_duplicates())

501

*Nota: Cabe destacar que hay muchos valores duplicados. Estos valores hay que tenerlos en cuenta en los resultados de las predicciónes.*

### Ejercicio 1:

> 1. ¿Cuáles son las 10 palabras más encontradas en correos con SPAM y en correos No SPAM? ¿Hay palabras en común? ¿Algunas llaman la atención?

In [9]:
# Agrupo por spam y me quedo solo con dicha fila.
grouped_spam = df_spam.groupby(by="spam", as_index=False).sum().iloc[1:2,:]

# Obtengo los índices de las columnas más grandes.
#   1 - Paso df a numpy.
#   2 - Lo paso a una dimensión.
#   3 - Lo ordeno (de forma inversa) y obtengo los 10 argumentos más grandes. 
lgt_items_index = np.argsort(grouped_spam.to_numpy().flatten())[-10:][::-1]

print(f'Correos SPAM (10 palabras más encontradas):')
# Simplemente filtro por ese vector de índices.
grouped_spam.iloc[:,lgt_items_index]

Correos SPAM (10 palabras más encontradas):


,word_freq_you,word_freq_your,word_freq_will,word_freq_free,word_freq_our,char_freq_!,word_freq_all,word_freq_mail,word_freq_email,word_freq_business
1,4105599,2502597,997100,939790,931799,931352,732080,635470,578759,521250


In [10]:
# Agrupo por NO spam y me quedo solo con dicha fila.
grouped_not_spam = df_spam.groupby(by="spam", as_index=False).sum().iloc[0:1,:]

# Obtengo los índices de las columnas más grandes.
#   1 - Paso df a numpy.
#   2 - Lo paso a una dimensión.
#   3 - Lo ordeno (de forma inversa) y obtengo los 10 argumentos más grandes. 
lgt_items_index = np.argsort(grouped_not_spam.to_numpy().flatten())[-10:][::-1]

print(f'Correos NO SPAM (10 palabras más encontradas):')
# Simplemente filtro por ese vector de índices.
grouped_not_spam.iloc[:,lgt_items_index]

Correos NO SPAM (10 palabras más encontradas):


,word_freq_you,word_freq_george,word_freq_hp,word_freq_will,word_freq_your,word_freq_hpl,word_freq_re,word_freq_edu,word_freq_address,word_freq_meeting
0,3541702,3527559,2496576,1495268,1223098,1204398,1159138,800669,681569,604460


### Ejercicio 2:

> 2. Separe el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba (70% y 30% respectivamente).

In [11]:
# X = df_spam.drop(columns="spam")
X = (df_spam.drop(columns="spam") * 100).astype(int)
y = df_spam["spam"]

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.3)

### Ejercicio 3:

> 3. Utilizando un clasificador de Bayes ingenuo, entrene con el conjunto de entrenamiento.

In [ ]:
# Escalamos los datos.
minmax_scaler = MinMaxScaler()
X_train_scaled = minmax_scaler.fit_transform(X_train)
X_test_scaled = minmax_scaler.transform(X_test)

# Lo transformamos en DataFrames
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

### Ejercicio 4:

> 4. Utilizando un clasificador de Regresión Logística, entrene con el conjunto de entrenamiento (en este caso, normalice los datos).

In [13]:
# Escalo los datos.
standar_scaler = StandardScaler()
X_train_scaled = standar_scaler.fit_transform(X_train)
X_test_scaled = standar_scaler.transform(X_test)

# Lo transformamos en DataFrames
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

In [16]:
classifier_balance = LogisticRegression(random_state = 0, class_weight="balanced", max_iter=10000)
classifier_balance.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=10000, random_state=0)

### Ejercicio 5:

> 5. Calcule la matriz de confusión del conjunto de evaluación para ambos modelos. ¿Qué tipo de error comete más cada modelo? ¿Cuál de los dos tipos de error crees que es más importante para este problema?

### Ejercicio 6:

> 6. Calcule la precisión y la recuperación de ambos modelos. Para cada métrica, ¿cuál es el mejor modelo? ¿Cómo se relacionan estas métricas con los tipos de errores analizados en el punto anterior? Expanda su respuesta.

### Ejercicio 7:

> 7. Obtenga la curva ROC y el AUC (Área Bajo la Curva ROC) de ambos modelos.